In [1]:
!pip install dfply
!pip install plotly
!pip install xhtml2pdf
import pandas as pd
import numpy as np
import math
from dfply import *
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import statistics 
import plotly.graph_objects as go
from scipy import stats
from scipy.stats import pearsonr
import plotly.graph_objects as go
import warnings
from scipy.stats import shapiro
from datetime import date, timedelta

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
players = pd.read_pickle("/content/drive/My Drive/Capstone/Data/Clean/Players_Clean.pkl")

In [0]:
trophies = pd.read_pickle("/content/drive/My Drive/Capstone/Data/Clean/trophies_Clean.pkl")

In [0]:
trophies = trophies.drop_duplicates()

In [0]:
markval = pd.read_pickle("/content/drive/My Drive/Capstone/Data/Clean/markval_detailed.pkl")

In [0]:
trophies["date"] = trophies.index
trophies.reset_index(inplace = True, drop = True)
trophies["year"] = trophies["date"].apply(lambda x : x.year)

In [0]:
players = players[players.sofifa_id.notnull()]

In [0]:
players.sofifa_id = players.sofifa_id.astype("int") 

In [0]:
trophies = trophies[trophies.sofifa_id.isin(players.sofifa_id.unique())]

In [0]:
trophies = pd.merge(trophies, players[["tm_id","sofifa_id"]], on = "sofifa_id")

In [0]:
trophies.drop("season",axis = "columns", inplace = True)
trophies.drop("date",axis = "columns", inplace = True)

In [0]:
trophies = pd.merge(trophies, markval, on = ["tm_id","year"])

In [0]:
trophies_mv = trophies >> group_by(X.tm_id, X.trophy) >> summarize(count = X.trophy.count(), mv = X.mv.mean())

# Trophies correlated variables.

In [15]:
corr, _ = pearsonr(trophies_mv["count"],trophies_mv["mv"])
print(corr)
fig = px.scatter(trophies_mv, "count","mv", color = "trophy", 
           title = "Trophies(Runner-up, Winner) vs Market Value", labels = {"count":"Trophies(Runner-up, Winner) won", "mv":"Market value"})
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))
fig.for_each_trace(lambda t: t.update(name=t.name.split("=")[1]))
fig.show()

0.43169419869696374


As we can see, in general the number of trophies won by the player is positively correlated to his market value. However, for the extreme cases the correlation is not strong, as young and expensive players mostly do not have many trophies, wherease players with huge amount of trophies are mostly old, meaning that they are not valued expensive.

In [0]:
trophies_age = trophies >> group_by(X.tm_id, X.trophy) >> summarize(count = X.trophy.count(), age = X.age.mean())

In [0]:
trophies_age = trophies_age.dropna()

In [18]:
corr, _ = pearsonr(trophies_age["count"],trophies_age["age"])
print(corr)
fig = px.scatter(trophies_age, "age","count", color = "trophy", 
           title = "Trophies(Runner-up, Winner) vs Age", labels = {"count":"Trophies(Runner-up, Winner) won", "age":"Player's age"})
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))
fig.for_each_trace(lambda t: t.update(name=t.name.split("=")[1]))
fig.show()

0.14080846317470785


As the plot shows there is a slight positive correlation between player's age and the number of trophies won by him, as older player's played for longer so had more opportunities for winning trophies.

In [0]:
trophies_cont = (trophies >> 
                 group_by(X.continent, X.trophy) >> 
                 summarize(count = X.trophy.count(), mv = X.mv.mean()))

# Trophies based on player's continent and position

In [20]:
fig = px.bar(trophies_cont, x = "trophy", y = "count", 
       facet_col = "continent", facet_col_wrap = 3, color = "trophy",
       title = "Winner and Runner-up trophies count by continents", 
       labels = {"count":"","trophy":""})
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))
fig.for_each_trace(lambda t: t.update(name=t.name.split("=")[1]))
fig.update_layout(showlegend = False)
fig.show()

As we can see in most of the continents except Europe and South America, the players win as much trophies as much as they lose finals. Like in almost all areas, european and South American players are again the best in terms of winning or getting 2nd places in tournaments.

In [0]:
trophies_pos = (trophies >> 
                 group_by(X.main_field_position, X.trophy) >> 
                 summarize(count = X.trophy.count(), mv = X.mv.mean()))

In [22]:
fig = px.bar(trophies_pos, x = "trophy", y = "count", 
       facet_col = "main_field_position", facet_col_wrap = 4, color = "trophy",
       title = "Winner and Runner-up trophies count by positions", 
       labels = {"count":"","trophy":""})
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))
fig.for_each_trace(lambda t: t.update(name=t.name.split("=")[1]))
fig.update_layout(showlegend = False)
fig.show()

According to the visualization, the deffenders earn highest number of trophies and goalkeepers the lowest. All the positions in general are more likely to win the trophies rathern than lose the final games or take 2nd place in the tournaments, but both events happen.